# Data preparation 
---
Avant importation, les bases de données ont été nettoyées de la manière suivante : 
- Suppression des colonnes inutiles dans le jeu athlete_events.csv, pour ne garder que les médailles, les années, les villes et les codes pays, ainsi que la distinction entre hiver et été pour pouvoir ne garder que le subset des JO d'été 

In [48]:
# importation du fichier et des packages 
import pandas as pd 
df = pd.read_csv("./datasets_clean/olympics_main/socio_athletes.csv", sep = ";")
# on ne garde que les JO d'été et on supprime la colonne saison ensuite
df = df.loc[df["Season"].str.contains("Summer")]
df = df.drop(columns = "Season")
# on groupe par année, médailles, et pays  
athlete_series = df.groupby(["NOC", "Year", "Medal"]).size()
df = athlete_series.to_frame(name = "Number of Medals").reset_index()
display(df)
# Pivot the dataframe to have separate columns for each medal type
df_pivot = df.pivot(index=["NOC", "Year"], columns="Medal", values="Number of Medals").reset_index().fillna(0)
df_pivot["Total"] = df_pivot["Bronze"] + df_pivot["Gold"] + df_pivot["Silver"]
display(df_pivot)
df_pivot.to_csv("./datasets_clean/olympics_main/country_year_medals.csv")

,NOC,Year,Medal,Number of Medals
0,AFG,2008,Bronze,1
1,AFG,2012,Bronze,1
2,AHO,1988,Silver,1
3,ALG,1984,Bronze,2
4,ALG,1992,Bronze,1
...,...,...,...,...
2804,ZIM,2004,Bronze,1
2805,ZIM,2004,Gold,1
2806,ZIM,2004,Silver,1
2807,ZIM,2008,Gold,1


Medal,NOC,Year,Bronze,Gold,Silver,Total
0,AFG,2008,1.0,0.0,0.0,1.0
1,AFG,2012,1.0,0.0,0.0,1.0
2,AHO,1988,0.0,0.0,1.0,1.0
3,ALG,1984,2.0,0.0,0.0,2.0
4,ALG,1992,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...
1270,ZAM,1984,1.0,0.0,0.0,1.0
1271,ZAM,1996,0.0,0.0,1.0,1.0
1272,ZIM,1980,0.0,15.0,0.0,15.0
1273,ZIM,2004,1.0,1.0,1.0,3.0
